# BeautifulSoup를 이용한 알라딘 베스트셀러 크롤링하기
---
파이썬(Python)을 이용한 크롤링에는 크게 두 가지 패키지가 사용됩니다. 동적 크롤링에 주로 사용되는 셀레니움(Selenium)과 정적 크롤링에 사용되는 뷰티풀수프(BeautifulSoup)죠. 전자는 동적 웹페이지(서버에 의해 통제되는 구조)의 경우 효율적으로 데이터를 크롤링할 수 있으며, 후자는 정적 웹페이지(태그가 그대로 표현되는 유형의 구조)의 경우 편리한 크롤링이 가능합니다. 이번에는 뷰티풀수프를 이용해서 알라딘 인터넷서점의 주간 베스트셀러 목록을 크롤링해 보려 합니다. 구조가 간단하고, 단순히 텍스트만 가져오면 되는 작업이라서 초보자 분들도 코드 몇 줄만 이해하면 따라할 수 있을겁니다.
</p></br></br>


만약 뷰티풀수프가 설치되지 않은 컴퓨터라면 터미널에 ```$pip install beautifulsoup4``` 설치 명령어를 입력해 최신 버전의 뷰티풀수프를 설치해 주시면 됩니다.
</p></br></br>


## 알라딘 베스트셀러 주소 알아보기
---
알라딘의 베스트셀러 주소는 ```https://aladin.co.kr/shop/common/wbest.aspx?BranchType=1``` 형식으로 구성되어 있습니다. 그런데, 여기서 페이지를 이동할 경우 1\~50위, 51\~100위, 151\~200위가 적혀있는 페이지는 각각 아래와 같이 URL이 표현됩니다.

* https://aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page=1&cnt=1000&SortOrder=1
* https://aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=2&CID=0&page=2&cnt=1000&SortOrder=1
* https://aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=3&CID=0&page=3&cnt=1000&SortOrder=1
 
위 주소의 패턴을 살펴보면, `page=X` (X는 임의의 숫자)로 나타나 있는 것을 알 수 있습니다. 여기서 페이지 번호를 하나씩 바꿔가면서 반복문을 돌리면, 원하는 항목만큼을 크롤링할 수 있겠네요.
</p></br></br>



In [1]:
# import package
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 알라딘 주간 베스트셀러 주소
page = 1  # 원하는 페이지 번호
url = f'https://aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page={page}&cnt=1000&SortOrder=1'

req = requests.get(url)
content = req.content
soup = BeautifulSoup(content, 'html.parser')

</p></br></br>


위와 같이 URL을 입력한 뒤, requests 패키지를 이용해 HTML 태그를 다운로드합니다. 다음으로는 BeautifulSoup를 이용해서 HTML 파싱을 해 줍니다. 이렇게 만들어진 soup 객체의 경우, HTML 태그의 기본적인 구조만 알고 있다면 어느 태그 안에 들어있는 텍스트를 가져오면 될지 선택할 수 있지요.
</p></br></br>


## 책 제목과 가격 파싱하기
---
알라딘의 책 제목은 하기 태그와 같이 bo3 클래스로 명명된 a 태그, 그 하위의 b 태그에 표기되어 있습니다. 예를 들면, 글 작성 시점에서 베스트셀러 1위인 `나는 메트로폴리탄 미술관의 경비원입니다` 책은 아래와 같이 표기되어 있습니다.
```html
<a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=328923887" class="bo3"><b>나는 메트로폴리탄 미술관의 경비원입니다</b></a>
```
</p></br></br>



In [2]:
# book name parsing
book_names = soup.select('a.bo3 b')
book_names[0].text

'나는 메트로폴리탄 미술관의 경비원입니다'

</p></br></br>


알라딘의 책 가격(실 구매가)은 책 제목과 같은 형태로 표기되어 있습니다. 예를 들어, 베스트셀러 1위 책의 가격은 아래와 같이 표기되어 있습니다. ss_p2 클래스로 명명된 span 태그, 그 하위의 span 태그에 표기되어 있지요. 원 단위가 따로 분리되어 있기 때문에, 자릿수 구분자인 반점 기호(,)만 제거해 주면 간편하게 이용할 수 있습니다.
```html
<span class="ss_p2"><b><span class="">15,750</span>원</b></span>
```
</p></br></br>



In [3]:
# book price parsing
book_prices = soup.select('span.ss_p2 span')
int(book_prices[0].text.replace(',',''))

15750

</p></br></br>


## 데이터 저장하기
---
이제 파이썬을 이용한 알라딘 베스트셀러 크롤링 데이터를 엑셀 파일로 저장해 보겠습니다. 반복문을 이용해서 한 페이지의 이름과 가격을 모두 리스트화 시켜준 다음, 판다스(Pandas)를 이용해서 데이터프레임(DataFrame) 형태로 변경 후 파일로 저장해 줍니다. 만약 2페이지 이상의 분량을 저장하고 싶다면, URL에 사용될 page 변수를 1에서 원하는 숫자까지 범위를 지정해 반복문을 만들어 주면 되겠습니다.
</p></br></br>



In [4]:
# one page parsing
book_names_parse = [name.text for name in book_names]
book_price_parse = [int(price.text.replace(',','')) for price in book_prices]

# make dataframe & file export
book_df = pd.DataFrame([book_names_parse, book_price_parse], index=['name', 'price']).T
book_df.to_excel('./result.xlsx', index=False)

book_df.head()

,name,price
0,나는 메트로폴리탄 미술관의 경비원입니다,15750
1,이처럼 사소한 것들,12420
2,월간순정 노자키 군 15 (한정판),12600
3,마흔에 읽는 쇼펜하우어,15300
4,ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩),17820
